In [ ]:
import pandas as pd

In [ ]:
import sys
import os

# 获取当前的工作目录的上一级目录
parent_dir = os.path.dirname(os.getcwd())

# 将上一级目录添加到 Python 路径中
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

# 现在你可以导入 mgtwr.sel 了
from mgtwr.sel import SearchMGTWRParameter
from mgtwr.model import MGTWR
import mgtwr
print(mgtwr.sel)

In [ ]:
# 定义你的文件路径
files = [os.path.join(parent_dir, 'data', f'alldatatable_{i}.csv') for i in range(6)]  # 根据你实际生成的文件数量调整

# 读取每个 CSV 文件并将它们合并为一个 DataFrame
data = pd.concat((pd.read_csv(f) for f in files))

In [ ]:
coords = data[['xcol', 'ycol']]
t = data[['timecol']]
X = data[['preliquid','surfwater','glemmerra','anag','presolid','canopystor']]
y = data[['GWSA']]

In [ ]:
import cProfile
import pstats
def test_func():
    sel_multi = SearchMGTWRParameter(coords, t, X, y, kernel='exponential', fixed=True,thread=14)
    sel_multi.search(multi_bw_min=[0.1], verbose=True, tol_multi=1.0e-4, time_cost=True)
    return sel_multi

profiler = cProfile.Profile()
profiler.enable()

sel_multi = None
try:
    sel_multi = test_func()
except KeyboardInterrupt:
    pass
finally:
    profiler.disable()
    stats = pstats.Stats(profiler).sort_stats('tottime')
    stats.print_stats()
    

In [ ]:
stats = pstats.Stats(profiler).sort_stats('cumtime')
stats.print_stats()

In [ ]:
mgtwr = MGTWR(coords, t, X, y, sel_multi, kernel='gaussian', fixed=True).fit()
print(mgtwr.R2)

In [ ]:
import os
print(os.getcwd())

In [ ]:
import pickle

# 保存变量
with open('mgtwr.pkl', 'wb') as f:
    pickle.dump(mgtwr, f)


# 加载变量
with open('mgtwr.pkl', 'rb') as f:
    loaded_mgtwr = pickle.load(f)

# 现在 loaded_mgtwr 和 mgtwr 是相同的
print(loaded_mgtwr.R2)